In [14]:
import tensorflow as tf
import numpy as np
CKPT_DIR = './vgg16ckpt/'
from dataloader import DataLoader
import cv2
import pickle
# Please download VGG16 pre-trained network from http://download.tensorflow.org/models/vgg_16_2016_08_28.tar.gz
# and then extract it to the project directory.

# Please also download the training features from https://drive.google.com/file/d/1AFffSGzjmmSxi2Qdjclv5m0Hah1v6P59/view?usp=sharing
# and the pre-trained SVM model from https://drive.google.com/file/d/1Rh_Kc-2hJxJV3pWoyg46PIcABy-5jHFN/view?usp=sharing

In [2]:
#Define the computational graph of VGG16 network.
class vgg16:
    def __init__(self, imgs):
        self.imgs = imgs
        self.convlayers()
        self.fc_layers()
        self.probs = tf.nn.softmax(self.fc8l)


    def convlayers(self):
        self.parameters = []

        # zero-mean input
        with tf.name_scope('preprocess') as scope:
            mean = tf.constant([123.68, 116.779, 103.939], dtype=tf.float32, shape=[1, 1, 1, 3], name='img_mean')
            images = self.imgs-mean

        with tf.name_scope('conv1') as sc:
            # conv1_1
            with tf.name_scope('conv1_1') as scope:
                kernel = tf.Variable(tf.truncated_normal([3, 3, 3, 64], dtype=tf.float32,
                                                         stddev=1e-1), name='weights')
                conv = tf.nn.conv2d(images, kernel, [1, 1, 1, 1], padding='SAME')
                biases = tf.Variable(tf.constant(0.0, shape=[64], dtype=tf.float32),
                                     trainable=True, name='biases')
                out = tf.nn.bias_add(conv, biases)
                self.conv1_1 = tf.nn.relu(out, name=scope)
                self.parameters += [kernel, biases]

            # conv1_2
            with tf.name_scope('conv1_2') as scope:
                kernel = tf.Variable(tf.truncated_normal([3, 3, 64, 64], dtype=tf.float32,
                                                         stddev=1e-1), name='weights')
                conv = tf.nn.conv2d(self.conv1_1, kernel, [1, 1, 1, 1], padding='SAME')
                biases = tf.Variable(tf.constant(0.0, shape=[64], dtype=tf.float32),
                                     trainable=True, name='biases')
                out = tf.nn.bias_add(conv, biases)
                self.conv1_2 = tf.nn.relu(out, name=scope)
                self.parameters += [kernel, biases]

            # pool1
            self.pool1 = tf.nn.max_pool(self.conv1_2,
                                   ksize=[1, 2, 2, 1],
                                   strides=[1, 2, 2, 1],
                                   padding='SAME',
                                   name='pool1')


        with tf.name_scope('conv2') as sc:
            # conv2_1
            with tf.name_scope('conv2_1') as scope:
                kernel = tf.Variable(tf.truncated_normal([3, 3, 64, 128], dtype=tf.float32,
                                                         stddev=1e-1), name='weights')
                conv = tf.nn.conv2d(self.pool1, kernel, [1, 1, 1, 1], padding='SAME')
                biases = tf.Variable(tf.constant(0.0, shape=[128], dtype=tf.float32),
                                     trainable=True, name='biases')
                out = tf.nn.bias_add(conv, biases)
                self.conv2_1 = tf.nn.relu(out, name=scope)
                self.parameters += [kernel, biases]

            # conv2_2
            with tf.name_scope('conv2_2') as scope:
                kernel = tf.Variable(tf.truncated_normal([3, 3, 128, 128], dtype=tf.float32,
                                                         stddev=1e-1), name='weights')
                conv = tf.nn.conv2d(self.conv2_1, kernel, [1, 1, 1, 1], padding='SAME')
                biases = tf.Variable(tf.constant(0.0, shape=[128], dtype=tf.float32),
                                     trainable=True, name='biases')
                out = tf.nn.bias_add(conv, biases)
                self.conv2_2 = tf.nn.relu(out, name=scope)
                self.parameters += [kernel, biases]

            # pool2
            self.pool2 = tf.nn.max_pool(self.conv2_2,
                                   ksize=[1, 2, 2, 1],
                                   strides=[1, 2, 2, 1],
                                   padding='SAME',
                                   name='pool2')


        with tf.name_scope('conv3') as sc:
            # conv3_1
            with tf.name_scope('conv3_1') as scope:
                kernel = tf.Variable(tf.truncated_normal([3, 3, 128, 256], dtype=tf.float32,
                                                         stddev=1e-1), name='weights')
                conv = tf.nn.conv2d(self.pool2, kernel, [1, 1, 1, 1], padding='SAME')
                biases = tf.Variable(tf.constant(0.0, shape=[256], dtype=tf.float32),
                                     trainable=True, name='biases')
                out = tf.nn.bias_add(conv, biases)
                self.conv3_1 = tf.nn.relu(out, name=scope)
                self.parameters += [kernel, biases]

            # conv3_2
            with tf.name_scope('conv3_2') as scope:
                kernel = tf.Variable(tf.truncated_normal([3, 3, 256, 256], dtype=tf.float32,
                                                         stddev=1e-1), name='weights')
                conv = tf.nn.conv2d(self.conv3_1, kernel, [1, 1, 1, 1], padding='SAME')
                biases = tf.Variable(tf.constant(0.0, shape=[256], dtype=tf.float32),
                                     trainable=True, name='biases')
                out = tf.nn.bias_add(conv, biases)
                self.conv3_2 = tf.nn.relu(out, name=scope)
                self.parameters += [kernel, biases]

            # conv3_3
            with tf.name_scope('conv3_3') as scope:
                kernel = tf.Variable(tf.truncated_normal([3, 3, 256, 256], dtype=tf.float32,
                                                         stddev=1e-1), name='weights')
                conv = tf.nn.conv2d(self.conv3_2, kernel, [1, 1, 1, 1], padding='SAME')
                biases = tf.Variable(tf.constant(0.0, shape=[256], dtype=tf.float32),
                                     trainable=True, name='biases')
                out = tf.nn.bias_add(conv, biases)
                self.conv3_3 = tf.nn.relu(out, name=scope)
                self.parameters += [kernel, biases]

            # pool3
            self.pool3 = tf.nn.max_pool(self.conv3_3,
                                   ksize=[1, 2, 2, 1],
                                   strides=[1, 2, 2, 1],
                                   padding='SAME',
                                   name='pool3')
        with tf.name_scope('conv4') as sc:
            # conv4_1
            with tf.name_scope('conv4_1') as scope:
                kernel = tf.Variable(tf.truncated_normal([3, 3, 256, 512], dtype=tf.float32,
                                                         stddev=1e-1), name='weights')
                conv = tf.nn.conv2d(self.pool3, kernel, [1, 1, 1, 1], padding='SAME')
                biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                                     trainable=True, name='biases')
                out = tf.nn.bias_add(conv, biases)
                self.conv4_1 = tf.nn.relu(out, name=scope)
                self.parameters += [kernel, biases]

            # conv4_2
            with tf.name_scope('conv4_2') as scope:
                kernel = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,
                                                         stddev=1e-1), name='weights')
                conv = tf.nn.conv2d(self.conv4_1, kernel, [1, 1, 1, 1], padding='SAME')
                biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                                     trainable=True, name='biases')
                out = tf.nn.bias_add(conv, biases)
                self.conv4_2 = tf.nn.relu(out, name=scope)
                self.parameters += [kernel, biases]

            # conv4_3
            with tf.name_scope('conv4_3') as scope:
                kernel = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,
                                                         stddev=1e-1), name='weights')
                conv = tf.nn.conv2d(self.conv4_2, kernel, [1, 1, 1, 1], padding='SAME')
                biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                                     trainable=True, name='biases')
                out = tf.nn.bias_add(conv, biases)
                self.conv4_3 = tf.nn.relu(out, name=scope)
                self.parameters += [kernel, biases]

            # pool4
            self.pool4 = tf.nn.max_pool(self.conv4_3,
                                   ksize=[1, 2, 2, 1],
                                   strides=[1, 2, 2, 1],
                                   padding='SAME',
                                   name='pool4')


        with tf.name_scope('conv5') as sc:
            # conv5_1
            with tf.name_scope('conv5_1') as scope:
                kernel = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,
                                                         stddev=1e-1), name='weights')
                conv = tf.nn.conv2d(self.pool4, kernel, [1, 1, 1, 1], padding='SAME')
                biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                                     trainable=True, name='biases')
                out = tf.nn.bias_add(conv, biases)
                self.conv5_1 = tf.nn.relu(out, name=scope)
                self.parameters += [kernel, biases]

            # conv5_2
            with tf.name_scope('conv5_2') as scope:
                kernel = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,
                                                         stddev=1e-1), name='weights')
                conv = tf.nn.conv2d(self.conv5_1, kernel, [1, 1, 1, 1], padding='SAME')
                biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                                     trainable=True, name='biases')
                out = tf.nn.bias_add(conv, biases)
                self.conv5_2 = tf.nn.relu(out, name=scope)
                self.parameters += [kernel, biases]

            # conv5_3
            with tf.name_scope('conv5_3') as scope:
                kernel = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,
                                                         stddev=1e-1), name='weights')
                conv = tf.nn.conv2d(self.conv5_2, kernel, [1, 1, 1, 1], padding='SAME')
                biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                                     trainable=True, name='biases')
                out = tf.nn.bias_add(conv, biases)
                self.conv5_3 = tf.nn.relu(out, name=scope)
                self.parameters += [kernel, biases]

            # pool5
            self.pool5 = tf.nn.max_pool(self.conv5_3,
                                   ksize=[1, 2, 2, 1],
                                   strides=[1, 2, 2, 1],
                                   padding='SAME',
                                   name='pool5')

    def fc_layers(self):
        # fc6
        with tf.name_scope('fc6') as scope:
            kernel = tf.Variable(tf.truncated_normal([7, 7, 512, 4096], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.pool5, kernel, [1, 1, 1, 1], padding='VALID')
            biases = tf.Variable(tf.constant(0.0, shape=[4096], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.fc6 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # fc7
        with tf.name_scope('fc7') as scope:
            kernel = tf.Variable(tf.truncated_normal([1, 1, 4096, 4096], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.fc6, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[4096], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.fc7 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # fc8
        with tf.name_scope('fc8') as scope:
            kernel = tf.Variable(tf.truncated_normal([1, 1, 4096, 1000], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.fc7, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[1000], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.fc8l = out
            self.fc8 = tf.nn.softmax(out, name=scope)
            self.parameters += [kernel, biases]

input_img = tf.placeholder(tf.float32,[None,224,224,3])

with tf.variable_scope('vgg_16') as scope:
    VGGModel = vgg16(imgs=input_img)

In [17]:
#Start a TF session, load the vgg_16.ckpt pre-trained model to the variables to initialize them.
sess = tf.Session()
vars_vgg = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='vgg_16')
saver_vgg = tf.train.Saver(max_to_keep=3, var_list=vars_vgg)
ckpt_vgg = tf.train.get_checkpoint_state(CKPT_DIR)
start_itr = 0
# if ckpt_vgg and ckpt_vgg.model_checkpoint_path:
saver_vgg.restore(sess, './vgg16ckpt/vgg_16.ckpt')
vggfc6feature = tf.squeeze(VGGModel.fc6, axis=[1, 2])
print("Model restored.")



INFO:tensorflow:Restoring parameters from ./vgg16ckpt/vgg_16.ckpt
Model restored...


# You may skip this step is you have downloaded the allfeatures.file binary.

In [31]:
# This step generates image features at the FC6 layer of VGG16.
# You may skip this step
loader = DataLoader()
features = []
lists = [loader.yes_image_name, loader.no_image_name]
# Run the graph in the session for every image crop.
for list in lists:
    for yesimg in list:
        img = cv2.imread(yesimg)
        img = np.expand_dims(img,axis=0)
        feature = sess.run(vggfc6feature,feed_dict={input_img:img})
        features.append(feature)

# allfeatures is a 2D numpy array of size 6084*4096, meaning 6084 pieces os of crops and each with a 4096-dim vector.
allfeatures = np.array(features)
allfeatures = np.squeeze(allfeatures)
print("allfeatures done")
with open ('allfeatures_py2.file','wb') as file:
    pickle.dump(allfeatures,file,protocol=2)

allfeatures done


In [32]:
from dataloader import DataLoader
from converter import send_crops_of_this_dispute_painting
import numpy as np
import pickle
from sklearn import svm
import os

# You may skip the following two steps if you have downloaded svmmodel.file

In [37]:
loader = DataLoader()
# Construct the Y-vector for training a simple SVM classifier. Y is a vector with the first 3525 elements being 1
# and the rest being 0. 
y = np.zeros([len(loader.yes_image_name) + len(loader.no_image_name)])
y[0:len(loader.yes_image_name)] = 1
allfeatures = None
with open ('allfeatures.file','rb') as file:
    allfeatures = pickle.load(file)

# Train the SVM classifier with extracted image features and their corresponding labels.
classifier = svm.SVC(probability=True) #
classifier.fit(allfeatures, y)
print("Training SVM Done.")

Training SVM Done.


In [39]:
with open ('svmmodel.file','wb') as file:
    pickle.dump(classifier,file)

In [ ]:
# You may load a pre-trained SVM model to object 'classifier'
classifier = svm.SVC()
with open ('svmmodel.file','rb') as file:
    classifier = pickle.load(file)

In [47]:
# Generate predictions for every crop that represents an image, and sum the score of all crops within a painting.
DISPUTE = [1, 7, 10, 20, 23, 25, 26]
for i in range(len(DISPUTE)):
    testdata = send_crops_of_this_dispute_painting(DISPUTE[i])
    print("painting number ",DISPUTE[i])
    allfeatures_thispainting = []
    for j in range(testdata.shape[0]):
        features_onecrop = sess.run(vggfc6feature,feed_dict={input_img:testdata[j:j+1,:,:,:]})
        allfeatures_thispainting.append(features_onecrop)
    pred = classifier.predict_proba(np.squeeze(np.array(allfeatures_thispainting)))
    scores = np.sum(pred,axis=0)
    print(scores)
    if scores[0] > scores[1]:
        print("Painting number ", DISPUTE[i] ," is fake")
    else:
        print("Painting number ", DISPUTE[i], " is real")

painting number  1
[ 153.86720668  186.13279332]
Painting number  1  is real
painting number  7
[ 110.81881602  239.18118398]
Painting number  7  is real
painting number  10
[  33.41622208  110.58377792]
Painting number  10  is real
painting number  20
[ 2.86952195  9.13047805]
Painting number  20  is real
painting number  23
[ 12.10396373  35.89603627]
Painting number  23  is real
painting number  25
[ 131.13854526  375.86145474]
Painting number  25  is real
painting number  26
[ 50.44111696  93.55888304]
Painting number  26  is real
